In [1]:
LOCAL=False
import os
import importlib

if LOCAL:
    path = '/Users/berk/Documents/research/nna/src/scripts/'
else:
    path = '/home/enis/projects/nna/src/scripts/'
os.chdir(path)

import teacher
from nna import weather

# import itertools
import pandas as pd


Output(layout=Layout(border='1px solid black'))

In [24]:
importlib.reload(teacher)
importlib.reload(weather)

version_tag='weather_V3'
config = teacher.setup(versiontag=version_tag)

weather_data_folder='/scratch/enis/data/nna/weather_data/2017_2020'
# weather_data_folder='/Users/berk/Downloads/NNA_files/weather_data/2017_2020'
root_path = '/scratch/enis/data/nna/labeling/samples'
config['split_out_path'] = f'{root_path}/{version_tag}/audio_'

## data-1: as many sampes as possible
config['new_dataset_path'] = f'{root_path}/{version_tag}/{version_tag}.csv'
config['dataset_version'] = 'W1'
# config['versiontag'] = 'yfitloiq-V1'
config['excell_label_headers']=['day_length','air_temp','snow_depth',
        'cloud_fraction','relative_humidity','runoff','rain_precip',
         'snow_precip','wind_direction','wind_speed']

# config['excell_label_headers']=['day_length','air_temp','snow_depth',
#         'cloud_fraction','relative_humidity','runoff','rain_precip',
#          'snow_precip','wind_direction','wind_speed',
#         'snow_blowing_ground','snow_blowing_air',]


config['upper_taxo_links']={}

FILE_PER_LOCATION=100
TIMESTAMPS_PER_ROW = 4
# print('total sample count to be produced:',file_per_location*len(short_ones.keys()))

# 40 Prudhoe or ANWR monitoring sites AND the Ivvavik sites
short_input_csv_headers = ['day_length','air_temp','snow_depth',
        'cloud_fraction','relative_humidity','runoff','rain_precip',
         'snow_precip','wind_direction','wind_speed']
# (year,month,day,hour,day_length,air_temp,snow_depth,
#                 cloud_fraction,relative_humidity,runoff,rain_precip,
#                     snow_precip,wind_direction,wind_speed)=row
# for Dalton and Dempster
long_input_csv_headers = ['day_length','air_temp','snow_depth',
        'cloud_fraction','relative_humidity','runoff','rain_precip',
         'snow_precip','total_precip','wind_direction','wind_speed',
        'snow_blowing_ground','snow_blowing_air']

excell_all_headers = [
    'data_version', 'Annotator', 'Site ID', 'File Name', 'Date', 'Start Time',
    'End Time', 'Length', 'Clip Path', 'Comments', 'weather_timestamp',
    'region', 'day_length', 'air_temp', 'snow_depth', 'cloud_fraction',
    'relative_humidity', 'runoff', 'rain_precip', 'snow_precip',
    'wind_direction', 'wind_speed'
]

# (year,month,day,hour,day_length,air_temp,snow_depth,
# cloud_fraction,relative_humidity,runoff,rain_precip,
# snow_precip,total_precip,wind_direction,wind_speed,
# snow_blowing_ground,snow_blowing_air)=row

short_locations=('prudhoe','ivvavik','anwr')
long_locations=('dalton','dempster')

if LOCAL:
    file_database = '/Users/berk/Documents/scratch/enis/data/nna/database/allFields_dataV10.pkl'
else:
    file_database = '/scratch/enis/data/nna/database/allFields_dataV10.pkl'

In [9]:
file_properties_df = pd.read_pickle(file_database)
station_csv = weather.csv_path_per_regloc(weather_data_folder)
station_years = weather.year_per_regloc(station_csv,file_properties_df)


In [25]:
### merge all data to see distributions
dataframes=[]
for (region,location),fname in station_csv.items():
    
    csv_reader,short = weather.load_rows(fname,region,short_locations,long_locations)
    pd_rows = weather.parse_rows(csv_reader,location,region,short,short_input_csv_headers,long_input_csv_headers)
    pd_rows = [weather.shift_row_timestamp_2_beginning_of_window(row) for row in pd_rows]

    # for this to work, get_random_rows needs to be fixed to use row not list
    # rows_picked = weather.get_random_rows(csv_reader,FILE_PER_LOCATION,station_years[(region,location)])

    print(region,location,len(pd_rows))

    selected_data=pd.DataFrame(pd_rows)
    dataframes.append(selected_data)

# unmatching columns will be filled with NaN
# ignore_index=True will create new index
selected_data=pd.concat(dataframes,ignore_index=True)

selected_data['rain_precip_mm'] = selected_data['rain_precip']*1000

ivvavik SINP03 35072


In [13]:
print('total sample count to be produced:',len(selected_data))

total sample count to be produced: 29600


In [5]:
buffer_seconds = int(3 * 3600 / (TIMESTAMPS_PER_ROW + 1)) + 1

new_dataset_csv, not_found_rows = teacher.generate_new_dataset(
    selected_data,
    config['versiontag'],
    config['split_out_path'],
    file_properties_df,
    config['upper_taxo_links'],
    config['dataset_version'],
    buffer=buffer_seconds,
    excell_label_headers= {},#config['excell_label_headers'],
    labels_thresholds= {},#config['labels_thresholds'],
    outputSuffix='.wav',
    dry_run=False,
    excell_labels_2_names= {}, #config['excell_labels_2_names'],
    stereo2mono=True,
    overwrite=False,
    sampling_rate=48000,
    label_row_by_threshold=False,
    print_logs=False,
)


start<first
start<first
start<first
start<first
start<first
start<first
start<first
start<first
start<first
start<first
start<first
start<first
start<first


In [ ]:
from copy import deepcopy
dataset_csv_backup = deepcopy(new_dataset_csv)
missing_csv_backup = deepcopy(not_found_rows)


for row in dataset_csv_backup:
    for k,v in row['weather_data'].items():
        if k not in ['location','TIMESTAMP']:
            row[k]=v
        row['weather_timestamp'] = row['weather_data']['TIMESTAMP']
    del row['weather_data']


for row in dataset_csv_backup:
    for k in list(row.keys()):
        if k not in excell_all_headers:
            del row[k]

dataset_csv_unique = {}
for row in dataset_csv_backup:
    dataset_csv_unique[row['Clip Path']] = row
dataset_csv_backup=list(dataset_csv_unique.values())

print(len(dataset_csv_backup))
teacher.write_csv(config['new_dataset_path'],
          dataset_csv_backup,
          fieldnames=excell_all_headers)

3700


In [30]:
dataset_csv_backup[0]

{'data_version': 'W1',
 'Annotator': 'weather_V1',
 'Site ID': 'SINP03',
 'File Name': '/tank/data/nna/real/ivvavik/SINP03/2019/SINP-03_20190513_231602.flac',
 'Start Time': '00:29:50.000000',
 'End Time': '00:30:00.000000',
 'Date': '05/14/2019',
 'Length': '00:00:10.000000',
 'Clip Path': PosixPath('/scratch/enis/data/nna/labeling/samples/weather_V1/audio_weather_V1/ivvavik/SINP03/SINP-03_20190513_231602_73m_48s__73m_58s.wav'),
 'Comments': '',
 'weather_timestamp': '2019-05-14_01:30:00',
 'region': 'ivvavik',
 'day_length': 20.2996368,
 'air_temp': 0.922241211,
 'snow_depth': 0.450530827,
 'cloud_fraction': 0.831612885,
 'relative_humidity': 84.5013275,
 'runoff': 0.0,
 'rain_precip': 3.24604343e-06,
 'snow_precip': 4.02112619e-06,
 'wind_direction': 291.185028,
 'wind_speed': 1.36716318}

In [68]:
len(dataset_csv)

3935

In [12]:
a=[row['Clip Path'] for row in dataset_csv]

In [13]:
len((a)),len(set(a))

(3811, 3700)

In [83]:
for row in dataset_csv:
    m = row['Clip Path']
    if m.exists():
        continue
    else:
        print(row)
        break

{'data_version': 'W2', 'Annotator': 'weather_V2', 'Site ID': '03', 'File Name': '/tank/data/nna/real/dalton/03/2019/S4A10437_20191009_172000.aac', 'Start Time': '17:20:00.000000', 'End Time': '17:20:10.000000', 'Date': '10/09/2019', 'Length': '00:00:10.000000', 'Clip Path': PosixPath('/scratch/enis/data/nna/labeling/samples/weather_V2/audio_weather_V2/dalton/03/S4A10437_20191009_172000_0m_0s__0m_10s.wav'), 'Comments': '', 'weather_timestamp': '2019-10-09_16:30:00', 'region': 'dalton', 'day_length': 9.85344791, 'air_temp': -0.127105713, 'snow_depth': 0.0195905175, 'cloud_fraction': 0.831612885, 'relative_humidity': 95.3483429, 'runoff': 0.0, 'rain_precip': 1.14912109e-05, 'snow_precip': 7.56504087e-05, 'total_precip': 8.71416196e-05, 'wind_direction': 192.373138, 'wind_speed': 1.04121852, 'snow_blowing_ground': 0.0, 'snow_blowing_air': 0.0787686035}
